## 영문 pdf들 간 유사도 분석하기
정확한 문서간 유사도를 추출해냈다기보단<br>
이후 쓰일 ai 모델의 학습 지표로 참고할 만 하다~ 정도일듯

---
#### **TF-IDF(Term Frequency - Inverse Document Frequency)**


TF : 단어 빈도
<br>IDF : 단어가 나타난 문서 수(의 역수)


---



### 1. package installation



In [20]:
import csv
import os
from pathlib import Path
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
import pandas as pd
from math import log

###2. txt to dataframe

In [23]:
#stopwords
nltk.download('stopwords')
print('checking stopwords . . .')
print(nltk.corpus.stopwords.words('english')[:20])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
checking stopwords . . .
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [24]:
path_dir = '../content/drive/MyDrive/dataset/wp_sample'
file_list = os.listdir(path_dir)
file_list

['Aave.txt',
 'IOTA.txt',
 'Bitcoin.txt',
 'Arweave.txt',
 'Stacks.txt',
 'Decentraland.txt',
 'TerraUSD.txt',
 'XRP.txt',
 'Wrapped Bitcoin.txt',
 'BitTorrent.txt',
 'Monero.txt',
 'Solana.txt',
 'USD Coin.txt',
 'NEM.txt',
 'Elrond.txt',
 'Ethereum.txt',
 'Avalanche.txt',
 'THETA.txt',
 'Klaytn.txt',
 'Zcash.txt',
 'asample.txt']

In [25]:
# data : coin 이름과 whitepaper text를 가진 dataframe
#폴더에 있는 whitepaper들을 하나씩 읽어서 data에 저장함
data = pd.DataFrame()
for file in file_list:
  filename = file.replace('.txt', '')
  file = path_dir + '/' + file
  txt = Path(file).read_text()
  txt = txt.replace('\n', '')
  data=data.append({'coin' : filename , 'whitepaper' : txt} , ignore_index=True)
data

,coin,whitepaper
0,Aave,Protocol WhitepaperV1.0wow@aave.comJanuary 202...
1,IOTA,at the beginning of the split to prevent an ho...
2,Bitcoin,Bitcoin: A Peer-to-Peer Electronic Cash System...
3,Arweave,Arweave LightpaperVersion 0.9Samuel WilliamsWi...
4,Stacks,Stacks 2.0 Apps and Smart Contracts for Bitco...
5,Decentraland,White paperDecentraland is a virtual reality p...
6,TerraUSD,Terra Money:Stability and AdoptionEvan Kereiak...
7,XRP,"Ripple Labs Inc, 2014The Ripple Protocol Conse..."
8,Wrapped Bitcoin,Wrapped Tokens A multi-institutional framewor...
9,BitTorrent,I BitTorrent (BTT) White Paperv0.8.7Feb. 2019B...


In [26]:
data_pr = data
data_pr['whitepaper']

0     Protocol WhitepaperV1.0wow@aave.comJanuary 202...
1     at the beginning of the split to prevent an ho...
2     Bitcoin: A Peer-to-Peer Electronic Cash System...
3     Arweave LightpaperVersion 0.9Samuel WilliamsWi...
4     Stacks 2.0 Apps and Smart Contracts for Bitco...
5     White paperDecentraland is a virtual reality p...
6     Terra Money:Stability and AdoptionEvan Kereiak...
7     Ripple Labs Inc, 2014The Ripple Protocol Conse...
8      Wrapped Tokens A multi-institutional framewor...
9     I BitTorrent (BTT) White Paperv0.8.7Feb. 2019B...
10    CryptoNote v 2.0Nicolas van SaberhagenOctober ...
11    Solana: A new architecture for a highperforman...
12    WhitepaperDocument Version 2.0  May 2018
3. Th...
13    Technical ReferenceNEMVersion 1.2.1February 23...
14    1ElrondA Highly Scalable Public Blockchain via...
15    Ethereum WhitepaperThis introductory paper was...
16    Scalable and Probabilistic Leaderless BFTConse...
17    AbstractThis whitepaper introduces the The

###3. text preprocessing

In [27]:
stop_word = set(stopwords.words('english'))

In [28]:
#특수문자 없앰
data_pr['whitepaper'] = data_pr['whitepaper'].str.replace("[^a-zA-Z\' ]","")
data_pr['whitepaper']

0     Protocol WhitepaperVwowaavecomJanuary Abstract...
1     at the beginning of the split to prevent an ho...
2     Bitcoin A PeertoPeer Electronic Cash SystemSat...
3     Arweave LightpaperVersion Samuel WilliamsWilli...
4     Stacks  Apps and Smart Contracts for BitcoinMu...
5     White paperDecentraland is a virtual reality p...
6     Terra MoneyStability and AdoptionEvan Kereiake...
7     Ripple Labs Inc The Ripple Protocol Consensus ...
8     Wrapped TokensA multiinstitutional framework f...
9     I BitTorrent BTT White PapervFeb BITTORRENT FO...
10    CryptoNote v Nicolas van SaberhagenOctober   I...
11    Solana A new architecture for a highperformanc...
12    WhitepaperDocument Version   May  The CENTRE T...
13    Technical ReferenceNEMVersion February  Conten...
14    ElrondA Highly Scalable Public Blockchain via ...
15    Ethereum WhitepaperThis introductory paper was...
16    Scalable and Probabilistic Leaderless BFTConse...
17    AbstractThis whitepaper introduces the The

In [29]:
#영어 소문자로 통일
data_pr['whitepaper'] = data_pr['whitepaper'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word not in (stop_word)]))

In [30]:
data_pr['whitepaper']

0     protocol whitepapervwowaavecomjanuary abstract...
1     beginning split prevent honest node eectively ...
2     bitcoin a peertopeer electronic cash systemsat...
3     arweave lightpaperversion samuel williamswilli...
4     stacks apps smart contracts bitcoinmuneeb aliw...
5     white paperdecentraland virtual reality platfo...
6     terra moneystability adoptionevan kereiakes do...
7     ripple labs inc the ripple protocol consensus ...
8     wrapped tokensa multiinstitutional framework t...
9     i bittorrent btt white papervfeb bittorrent fo...
10    cryptonote v nicolas van saberhagenoctober int...
11    solana a new architecture highperformance bloc...
12    whitepaperdocument version may the centre team...
13    technical referencenemversion february content...
14    elronda highly scalable public blockchain via ...
15    ethereum whitepaperthis introductory paper ori...
16    scalable probabilistic leaderless bftconsensus...
17    abstractthis whitepaper introduces theta n

###4. calculating similarites

In [31]:
#TfidfVectorizer 사용했음
#2번이하, 20번 이상 나오는 단어들은 제외
#ngram range는 (2, 3)

v = TfidfVectorizer(max_df = 20, min_df=2, ngram_range=(2,3))
x = v.fit_transform(data_pr['whitepaper'])

In [32]:
v.vocabulary_

{'document describes': 894,
 'protocol architecture': 2326,
 'governance the': 1206,
 'provide liquidity': 2373,
 'instead rely': 1401,
 'this enables': 3051,
 'based state': 266,
 'figure belowfigure': 1092,
 'amount funds': 166,
 'total amount': 3187,
 'greater value': 1208,
 'considered the': 622,
 'the amount': 2901,
 'figure shows': 1095,
 'shows example': 2640,
 'price fluctuations': 2246,
 'price ofthe': 2247,
 'point time': 2197,
 'last update': 1507,
 'updated every': 3384,
 'every time': 1018,
 'rate the': 2418,
 'in case': 1327,
 'case multiple': 492,
 'implementation protocol': 1316,
 'state every': 2765,
 'basic logic': 268,
 'provide high': 2372,
 'high level': 1254,
 'level information': 1533,
 'contract uses': 654,
 'advanced features': 81,
 'further details': 1166,
 'contract holds': 646,
 'take account': 2856,
 'account cost': 22,
 'described section': 827,
 'onchain governance': 2009,
 'important make': 1320,
 'make distinction': 1595,
 'creation multiple': 717,
 'ta

In [33]:
similarities = (x * x.T)

In [34]:
#편리한 계산을 위해 자기자신과의 유사도(=1)을 0으로 바꿔줌

result = similarities.toarray()
result = np.where(result>=0.99, 0, result)
result

array([[0.        , 0.01262067, 0.02107038, 0.01047475, 0.02825023,
        0.06247754, 0.06494124, 0.01671901, 0.06499681, 0.02781538,
        0.02896444, 0.02133854, 0.04988365, 0.01604145, 0.05741695,
        0.06791296, 0.02133545, 0.03984112, 0.08442715, 0.03161108,
        0.04355322],
       [0.01262067, 0.        , 0.05098265, 0.02968656, 0.03980308,
        0.00306547, 0.00106959, 0.03677921, 0.03448274, 0.01143045,
        0.01501822, 0.02828732, 0.00569396, 0.03135666, 0.01347045,
        0.05728656, 0.05539295, 0.02699942, 0.0183255 , 0.01329117,
        0.        ],
       [0.02107038, 0.05098265, 0.        , 0.03702192, 0.02767939,
        0.02286468, 0.02958375, 0.02918639, 0.03959234, 0.01371036,
        0.12584576, 0.03787507, 0.03638059, 0.0876312 , 0.0700522 ,
        0.12303095, 0.08951384, 0.12074147, 0.07072701, 0.09450953,
        0.        ],
       [0.01047475, 0.02968656, 0.03702192, 0.        , 0.06363315,
        0.08053304, 0.02777465, 0.03444597, 0.0211061

In [35]:
result = pd.DataFrame(result)
idx_copy_max = result.idxmax(axis=1)
result

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0.000000,0.012621,0.021070,0.010475,0.028250,0.062478,0.064941,0.016719,0.064997,0.027815,0.028964,0.021339,0.049884,0.016041,0.057417,0.067913,0.021335,0.039841,0.084427,0.031611,0.043553
1,0.012621,0.000000,0.050983,0.029687,0.039803,0.003065,0.001070,0.036779,0.034483,0.011430,0.015018,0.028287,0.005694,0.031357,0.013470,0.057287,0.055393,0.026999,0.018326,0.013291,0.000000
2,0.021070,0.050983,0.000000,0.037022,0.027679,0.022865,0.029584,0.029186,0.039592,0.013710,0.125846,0.037875,0.036381,0.087631,0.070052,0.123031,0.089514,0.120741,0.070727,0.094510,0.000000
3,0.010475,0.029687,0.037022,0.000000,0.063633,0.080533,0.027775,0.034446,0.021106,0.039391,0.037186,0.056857,0.036588,0.087739,0.099317,0.109142,0.046277,0.061108,0.101839,0.026623,0.000000
4,0.028250,0.039803,0.027679,0.063633,0.000000,0.055647,0.110144,0.044953,0.088140,0.030183,0.028129,0.061119,0.145203,0.027744,0.205650,0.143461,0.039340,0.111964,0.289859,0.026501,0.000000
5,0.062478,0.003065,0.022865,0.080533,0.055647,0.000000,0.033766,0.032160,0.061732,0.041265,0.048969,0.016585,0.082897,0.018740,0.046976,0.073900,0.036095,0.065077,0.100070,0.024597,0.007530
6,0.064941,0.001070,0.029584,0.027775,0.110144,0.033766,0.000000,0.016815,0.084541,0.023076,0.046042,0.040211,0.130425,0.023518,0.045117,0.068729,0.031998,0.037109,0.074211,0.021615,0.043803
7,0.016719,0.036779,0.029186,0.034446,0.044953,0.032160,0.016815,0.000000,0.023462,0.011216,0.033693,0.043085,0.021000,0.051723,0.070703,0.055790,0.144162,0.081221,0.048061,0.022250,0.000000
8,0.064997,0.034483,0.039592,0.021106,0.088140,0.061732,0.084541,0.023462,0.000000,0.022426,0.017948,0.034461,0.072170,0.028216,0.060188,0.063290,0.019466,0.037959,0.110170,0.019998,0.016994
9,0.027815,0.011430,0.013710,0.039391,0.030183,0.041265,0.023076,0.011216,0.022426,0.000000,0.023927,0.029174,0.145376,0.028785,0.023156,0.048889,0.031020,0.064624,0.173848,0.016430,0.029898


In [36]:
#최종 결과 csv로 만들 애들을 feature dataframe으로 모았음
#feature column들은 밑에서 설명

feature = pd.DataFrame(data = data_pr['coin'])  

In [37]:
feature['copy_max'] = result.max(axis=1, numeric_only=True)
feature['idx_copy_max'] = idx_copy_max
feature

,coin,copy_max,idx_copy_max
0,Aave,0.084427,18
1,IOTA,0.057287,15
2,Bitcoin,0.125846,10
3,Arweave,0.109142,15
4,Stacks,0.289859,18
5,Decentraland,0.100070,18
6,TerraUSD,0.130425,12
7,XRP,0.144162,16
8,Wrapped Bitcoin,0.110170,18
9,BitTorrent,0.173848,18


In [38]:
coin_name = feature['coin']
feature['coin_copy_max'] = feature['idx_copy_max'].apply(lambda x : coin_name[x])

In [39]:
cnt = result.apply(lambda x : x.value_counts())
cnt = cnt.iloc[cnt.index>=0.15]
cnt = cnt.fillna(0)

In [40]:
cnt_copy = cnt.sum(axis = 0)
cnt_copy

0     0.0
1     0.0
2     0.0
3     0.0
4     2.0
5     0.0
6     0.0
7     0.0
8     0.0
9     1.0
10    2.0
11    0.0
12    1.0
13    2.0
14    5.0
15    2.0
16    1.0
17    1.0
18    5.0
19    2.0
20    0.0
dtype: float64

In [41]:
feature['cnt_copy'] = cnt_copy

###5. results


---
feature 설명<br>
* coin : 코인 이름<br>
* coin_max : 가장 높게 나온 유사도 수치<br>
* idx_copy_max : white paper 유사도가 가장 높게 나온 코인의 index<br>
* coin_copy_max : white paper 유사도가 가장 높게 나온 코인의 이름<br>
* cnt_copy : white paper 유사도가 15%이상 나온 문서의 개수<br>


---




In [42]:
feature.sort_values(by=['cnt_copy'], ascending=False)

,coin,copy_max,idx_copy_max,coin_copy_max,cnt_copy
18,Klaytn,0.289859,4,Stacks,5.0
14,Elrond,0.261557,18,Klaytn,5.0
10,Monero,0.261938,19,Zcash,2.0
19,Zcash,0.261938,10,Monero,2.0
4,Stacks,0.289859,18,Klaytn,2.0
15,Ethereum,0.190001,18,Klaytn,2.0
13,NEM,0.246231,19,Zcash,2.0
12,USD Coin,0.177483,18,Klaytn,1.0
17,THETA,0.167898,14,Elrond,1.0
16,Avalanche,0.153925,14,Elrond,1.0


In [43]:
feature.to_csv('feature_wp.csv', sep=',')